10.1 GroupBy Mechanics

In [4]:
import pandas as pd
import numpy as np


df = pd.DataFrame({'key1': ['a', 'a', 'b', 'b', 'a'],
                   'key2': ['one', 'two', 'one', 'two', 'one'],
                   'data1': np.random.randn(5),
                   'data2': np.random.randn(5)})
df
grouped = df['data1'].groupby(df['key1'])
grouped
grouped.mean()


key1
a    0.240843
b    0.104298
Name: data1, dtype: float64

In [5]:
means = df['data1'].groupby([df['key1'], df['key2']]).mean()
means

key1  key2
a     one     0.549970
      two    -0.377410
b     one     0.488064
      two    -0.279469
Name: data1, dtype: float64

In [6]:
means.unstack()

key2,one,two
key1,,
a,0.549970,-0.377410
b,0.488064,-0.279469


In [7]:
states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])
years = np.array([2005, 2005, 2006, 2005, 2006])

df['data1'].groupby([states, years]).mean()

California  2005   -0.377410
            2006    0.488064
Ohio        2005    0.409349
            2006    0.001773
Name: data1, dtype: float64

In [13]:
df.groupby('key1').mean(numeric_only=True)

,data1,data2
key1,,
a,0.240843,-1.694155
b,0.104298,1.012113


In [11]:
df.groupby(['key1', 'key2']).mean()

data1     data2
key1 key2                    
a    one   0.549970 -1.010480
     two  -0.377410 -3.061505
b    one   0.488064  2.202667
     two  -0.279469 -0.178441

In [14]:
df.groupby(['key1', 'key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

In [15]:
# Iterating Over Groups

for name, group in df.groupby('key1'):
    print(name)
    print(group)

a
  key1 key2     data1     data2
0    a  one  1.098167 -0.835256
1    a  two -0.377410 -3.061505
4    a  one  0.001773 -1.185705
b
  key1 key2     data1     data2
2    b  one  0.488064  2.202667
3    b  two -0.279469 -0.178441


In [17]:
for (k1, k2), group in df.groupby(['key1', 'key2']):
    print((k1, k2))
    print(group)

('a', 'one')
  key1 key2     data1     data2
0    a  one  1.098167 -0.835256
4    a  one  0.001773 -1.185705
('a', 'two')
  key1 key2    data1     data2
1    a  two -0.37741 -3.061505
('b', 'one')
  key1 key2     data1     data2
2    b  one  0.488064  2.202667
('b', 'two')
  key1 key2     data1     data2
3    b  two -0.279469 -0.178441


In [18]:
pieces = dict(list(df.groupby('key1')))
pieces['b']

,key1,key2,data1,data2
2,b,one,0.488064,2.202667
3,b,two,-0.279469,-0.178441


In [20]:
df.dtypes
for dtype, group in grouped:
    print(dtype)
    print(group)

a
0    1.098167
1   -0.377410
4    0.001773
Name: data1, dtype: float64
b
2    0.488064
3   -0.279469
Name: data1, dtype: float64


In [26]:
# Selecting a Column or Subset of Columns

df.groupby(['key1', 'key2'])[['data2']].mean()

data2
key1 key2          
a    one  -1.010480
     two  -3.061505
b    one   2.202667
     two  -0.178441

In [29]:
s_grouped = df.groupby(['key1', 'key2'])['data2']
s_grouped
s_grouped.mean()

key1  key2
a     one    -1.010480
      two    -3.061505
b     one     2.202667
      two    -0.178441
Name: data2, dtype: float64

In [21]:
# Grouping with Dicts and Series

people = pd.DataFrame(np.random.randn(5, 5),
                      columns=['a', 'b', 'c', 'd', 'e'],
                      index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])
people.iloc[2:3, [1, 2]] = np.nan
people

,a,b,c,d,e
Joe,0.497707,-1.869093,0.163027,-0.060932,0.719777
Steve,-0.134266,0.350190,-0.149868,-0.251269,-1.036591
Wes,0.116678,NaN,NaN,-0.352340,1.069841
Jim,1.807934,-0.489560,0.562145,-0.135978,-1.046192
Travis,0.131337,-0.264420,0.061621,0.684875,-0.060339


In [23]:
mapping = {'a': 'red', 'b': 'red', 'c': 'blue', 'd': 'blue', 'e': 'red', 'f': 'orange'}
by_column = people.T.groupby(mapping)
by_column.sum()

,Joe,Steve,Wes,Jim,Travis
blue,0.102095,-0.401137,-0.352340,0.426167,0.746496
red,-0.651609,-0.820667,1.186519,0.272183,-0.193422


In [25]:
map_series = pd.Series(mapping)
map_series
people.T.groupby(map_series).count()

,Joe,Steve,Wes,Jim,Travis
blue,2,2,1,2,2
red,3,3,2,3,3


In [30]:
# Grouping with Functions

people.groupby(len).sum()

,a,b,c,d,e
3,2.422320,-2.358653,0.725171,-0.549249,0.743426
5,-0.134266,0.350190,-0.149868,-0.251269,-1.036591
6,0.131337,-0.264420,0.061621,0.684875,-0.060339


In [31]:
key_list = ['one', 'one', 'one', 'two', 'two']
people.groupby([len, key_list]).min()

a         b         c         d         e
3 one  0.116678 -1.869093  0.163027 -0.352340  0.719777
  two  1.807934 -0.489560  0.562145 -0.135978 -1.046192
5 one -0.134266  0.350190 -0.149868 -0.251269 -1.036591
6 two  0.131337 -0.264420  0.061621  0.684875 -0.060339

In [32]:
# Grouping by Index Levels

columns = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP', 'JP'],
                                     [1, 3, 5, 1, 3]],
                                     names=['city', 'tenor'])
hier_df = pd.DataFrame(np.random.randn(4, 5), columns=columns)
hier_df

city         US                            JP          
tenor         1         3         5         1         3
0      0.474790 -1.294720  0.857986 -1.352184 -0.352745
1      1.318117  0.326768  1.865385  0.637871 -1.397068
2     -0.565447 -0.755753  0.111416 -0.305201 -0.310356
3      0.233398 -0.185457  0.769585  0.206446 -0.147800

In [35]:
hier_df.T.groupby(level='city').count()

,0,1,2,3
city,,,,
JP,2,2,2,2
US,3,3,3,3
